In [1]:
!pip install selenium

# 데이터 크롤링

In [2]:
## 필요한 패키지들 import
# 셀레니움 크롤링 시 필요한 패키지
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests as req
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 이벤트 제어 라이브러리 (클릭, 드래그 앤 드롭 등등)
from selenium.webdriver.common.action_chains import ActionChains

# 크롤링 간 대기시간을 설정해, 오류를 방지하기 위함.
import time

In [3]:
# 가상 브라우저 실행 및 사이트 띄우기
driver = wb.Chrome()
driver.get("https://nfds.go.kr/stat/general.do")

In [87]:
# 주의사항 : 지역 > 행정구역 > 서울특별시까지는 직접 눌러야 한다.

# 반복문의 전체적인 과정 
# 1. 검색창의 내용을 지운다.
# 2. 2018-01-01부터 시작날짜, 종료날짜에 똑같이 입력해준다. 
# 3. 검색하기를 누르고, 원하는 구 - 화재건수만 추출한다.
# 4. 이걸 2022-12-31까지 해서 원하는 구 리스트를 만든다.
# 5. 이를 각 구마다 반복한다.

# 반복문 생성
# 1. 각 구의 이름을 딴 리스트를 만든다 ex) 강남구 = []
# 2. 필요한 요소는 날짜, 발생횟수임.
# 3. 그래서 일단 화재발생일수만 뽑고, 나중에 인덱스를 날짜로 변경한다.(어차피 날짜 순서대로의 데이터라서.)

In [166]:
# 검색 전, 검색 칸의 내용을 지워주는 코드
# 일단 검색칸 중 시작날짜에 있는 내용을 지워준다.
date_start = driver.find_element(By.NAME, "datePicker1s")
date_start.clear()

# 종료날짜도 똑같이 지워준다.
date_end = driver.find_element(By.NAME, "datePicker1e")
date_end.clear()

In [8]:
# 날짜 리스트 저장
date = pd.date_range(start='20180101', end='20221231')

date_list = date.strftime("%Y-%m-%d").tolist()

In [167]:
# 시작날짜, 종료날짜에 원하는 날짜 넣는 코드 ex. 2018-01-01
date_start.send_keys(date_list[0]) ; date_end.send_keys(date_list[0])

In [168]:
# 검색하기 누르는 코드
# 객체 생성
actions = ActionChains(driver)

# 검색하기 위치 지정
# searchItem > div.search-words > div.search-option > div.page-search > a:nth-child(1) > button 
elm_click = driver.find_element(By.CSS_SELECTOR,
                                "#searchItem > div.search-words > div.search-option > div.page-search > a:nth-child(1) > button")

# 검색하기 클릭
elm_click.click()

In [169]:
# 예시 : 2018-01-01의 화재발생횟수를 가져와보기
# 결과가 webelement 형태로 나와서, .text를 붙여 text로 바꿔준다.
# ex. 강남구 화재건수 : #grdViewer > table > tbody > tr:nth-child(4) > td:nth-child(2)
driver.find_element(By.CSS_SELECTOR,
                    "#grdViewer > table > tbody > tr:nth-child(4) > td:nth-child(2)").text

'1'

In [155]:
# 원하는 정보를 크롤링해오는 코드
# 지역구 리스트를 만들고, 화재 발생 건수를 한번에 받아 데이터 프레임으로 만들 예정

# 먼저 지역구 리스트부터 만든다.
region_list = ["강남구", "강동구", "강북구", "강서구", "관악구", "광진구",
               "구로구", "금천구", "노원구", "도봉구", "동대문구", "동작구",
              "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구",
              "양천구", "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"]

# 화재 발생 건수를 담을 빈 리스트를 만든다
fire_counts = []

# 반복문 시작
for date in date_list :
    # 검색칸 중 시작날짜에 있는 내용을 지워준다.
    element_start = driver.find_element(By.NAME, "datePicker1s")
    element_start.clear()

    # 종료날짜도 똑같이 지워준다.
    element_end = driver.find_element(By.NAME, "datePicker1e")
    element_end.clear()
    
    # 시작날짜, 종료날짜에 처음부터 끝까지의 날짜 넣기
    date_start.send_keys(date) ; date_end.send_keys(date)
    
    # 검색 실행 > 데이터가 나온다.
    elm_click.click()
    
    # 클릭 이후 기다리면서 에러 방지하기
    # 기다려 주지 않으면, 날짜가 바뀌면서 정보가 바뀌는데, 그 속도가 컴퓨터의 속도보다 한참 느려 에러가 발생한다.
    time.sleep(1)
    
    # 이제 지역구 별로 화재발생 건수를 크롤링해온다.
    # f-string을 이용하여 선택자의 내부 숫자만 바꿀 예정인데, 이를 위해 선택자의 수를 파악할 필요가 있다.
    
    # 강남구 화재건수 : #grdViewer > table > tbody > tr:nth-child(4) > td:nth-child(2)
    # 중랑구 화재건수 : #grdViewer > table > tbody > tr:nth-child(28) > td:nth-child(2)
    # 이 상황에서, 4에서 28까지의 숫자를 고려하여 for 반복문을 설정해 준다.
    for i in range(4, 28 + 1) : # 4 ~ 28 = 강남구 ~ 중랑구
        fire_count = int(driver.find_element(By.CSS_SELECTOR, 
                                             f"#grdViewer > table > tbody > tr:nth-child({i}) > td:nth-child(2)").text)
        fire_counts.append(fire_count)
    

In [156]:
# 위 크롤링 정보를 바탕으로 데이터프레임 생성
# 화재건수 데이터의 형태 변경
import numpy as np

# 열에는 지역구 이름 쓸것이므로, region_list에 맞게 재배열
fire_counts = np.array(fire_counts).reshape(-1, len(region_list))

df = pd.DataFrame(fire_counts, columns=region_list, index=date_list) ; df

강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  도봉구  ...  성동구  성북구  \
2018-01-01    1    1    2    0    1    1    0    2    0    1  ...    2    0   
2018-01-02    0    0    0    1    1    0    1    0    0    0  ...    0    0   
2018-01-03    5    0    0    2    3    0    1    1    0    0  ...    0    1   
2018-01-04    0    0    0    0    0    0    0    0    1    0  ...    0    1   
2018-01-05    2    2    1    2    0    1    0    1    0    1  ...    0    0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2022-12-27    1    1    0    2    1    0    1    1    0    0  ...    1    1   
2022-12-28    2    0    1    1    0    0    0    0    0    0  ...    0    3   
2022-12-29    1    0    1    1    0    1    2    1    1    0  ...    1    0   
2022-12-30    2    0    0    0    2    0    0    0    0    0  ...    1    1   
2022-12-31    1    1    2    0    1    1    2    2    0    0  ...    0    1   

            송파구  양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
2018-01-01    2    0     1    0    0    0   2    1  
2018-01-02    0    1     1    0    1    1   0    1  
2018-01-03    4    0     0    0    0    1   0    1  
2018-01-04    3    0     1    1    0    2   0    2  
2018-01-05    0    0     0    0    1    2   0    0  
...         ...  ...   ...  ...  ...  ...  ..  ...  
2022-12-27    1    3     0    1    0    0   0    0  
2022-12-28    0    0     0    3    0    1   0    0  
2022-12-29    2    0     0    3    1    0   0    1  
2022-12-30    2    0     2    1    1    0   2    0  
2022-12-31    2    0     0    2    1    0   1    1  

[1826 rows x 25 columns]

In [163]:
# 결과값 csv 파일로 저장
# 디폴트값인 utf-8로 저장시 파일 깨짐 > 한글 인코딩 방식인 cp949 사용 (euc-kr도 깨짐)
df.to_csv("화재발생건수.csv", encoding='cp949')

# 데이터 전처리
- 날짜 컬럼의 컬럼명 변경
- 각 구별로 작은 데이터프레임으로 나누기

In [56]:
# csv 파일을 다시 불러오기
df = pd.read_csv("화재발생건수.csv", encoding = 'cp949')

In [57]:
# 날짜 컬럼의 컬럼명을 Day로 변경
df.rename(columns = {"Unnamed: 0" : "day"}, inplace = True) ; df

day  강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  ...  성동구  성북구  \
0     2018-01-01    1    1    2    0    1    1    0    2    0  ...    2    0   
1     2018-01-02    0    0    0    1    1    0    1    0    0  ...    0    0   
2     2018-01-03    5    0    0    2    3    0    1    1    0  ...    0    1   
3     2018-01-04    0    0    0    0    0    0    0    0    1  ...    0    1   
4     2018-01-05    2    2    1    2    0    1    0    1    0  ...    0    0   
...          ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
1821  2022-12-27    1    1    0    2    1    0    1    1    0  ...    1    1   
1822  2022-12-28    2    0    1    1    0    0    0    0    0  ...    0    3   
1823  2022-12-29    1    0    1    1    0    1    2    1    1  ...    1    0   
1824  2022-12-30    2    0    0    0    2    0    0    0    0  ...    1    1   
1825  2022-12-31    1    1    2    0    1    1    2    2    0  ...    0    1   

      송파구  양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
0       2    0     1    0    0    0   2    1  
1       0    1     1    0    1    1   0    1  
2       4    0     0    0    0    1   0    1  
3       3    0     1    1    0    2   0    2  
4       0    0     0    0    1    2   0    0  
...   ...  ...   ...  ...  ...  ...  ..  ...  
1821    1    3     0    1    0    0   0    0  
1822    0    0     0    3    0    1   0    0  
1823    2    0     0    3    1    0   0    1  
1824    2    0     2    1    1    0   2    0  
1825    2    0     0    2    1    0   1    1  

[1826 rows x 26 columns]

In [71]:
# df의 인덱스를 day로 설정하고, 기존에 있던 day라는 컬럼 제거
df = df.set_index(keys = df["day"])
df = df.drop("day", axis = 1) ; df

강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  도봉구  ...  성동구  성북구  \
day                                                           ...             
2018-01-01    1    1    2    0    1    1    0    2    0    1  ...    2    0   
2018-01-02    0    0    0    1    1    0    1    0    0    0  ...    0    0   
2018-01-03    5    0    0    2    3    0    1    1    0    0  ...    0    1   
2018-01-04    0    0    0    0    0    0    0    0    1    0  ...    0    1   
2018-01-05    2    2    1    2    0    1    0    1    0    1  ...    0    0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2022-12-27    1    1    0    2    1    0    1    1    0    0  ...    1    1   
2022-12-28    2    0    1    1    0    0    0    0    0    0  ...    0    3   
2022-12-29    1    0    1    1    0    1    2    1    1    0  ...    1    0   
2022-12-30    2    0    0    0    2    0    0    0    0    0  ...    1    1   
2022-12-31    1    1    2    0    1    1    2    2    0    0  ...    0    1   

            송파구  양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
day                                                 
2018-01-01    2    0     1    0    0    0   2    1  
2018-01-02    0    1     1    0    1    1   0    1  
2018-01-03    4    0     0    0    0    1   0    1  
2018-01-04    3    0     1    1    0    2   0    2  
2018-01-05    0    0     0    0    1    2   0    0  
...         ...  ...   ...  ...  ...  ...  ..  ...  
2022-12-27    1    3     0    1    0    0   0    0  
2022-12-28    0    0     0    3    0    1   0    0  
2022-12-29    2    0     0    3    1    0   0    1  
2022-12-30    2    0     2    1    1    0   2    0  
2022-12-31    2    0     0    2    1    0   1    1  

[1826 rows x 25 columns]

In [78]:
# 각 구별 csv 파일을 저장해야 하는데, 기상 데이터와 합치기 위해선 컬럼명을 "발생 건수"로 바꿔줘야 한다.
# 그렇기에 rename 함수를 써서 구 이름을 바꿔주고, 그것을 바로 csv 파일로 저장해서 보내준다.

# 예시 : 강남구 데이터
df.rename(columns = {"강남구" : "발생 건수"})[["발생 건수"]].to_csv("강남구 화재발생건수.csv", encoding = "cp949")

In [97]:
# 예시 코드 방식을 반복문으로 써서, 각 구별 csv를 다운받는다.
for region in region_list :
    df.rename(columns = {f"{region}" : "발생 건수"})[["발생 건수"]].to_csv(f"{region} 화재발생건수.csv", encoding = "cp949")
    
# 결과 : 강남구 화재발생건수 ~ 중랑구 화재발생건수.csv

## 이 아래는 서울시 전체를 다룬다 > 이제 필요없음

In [95]:
# 특정 날짜에서 일어난 모든 화재 건수를 합쳐 "전체화재발생건수" 라는 컬럼을 만들 예정이다.
# 아래 예시는, 0번째 열(2018-01-01)에 포함된 모든 화재발생건수의 예시다.
sum(df.iloc[0,])

19

In [151]:
# 전 날짜에 걸친 화재발생건수를 한 리스트에 담아, 그 리스트를 서울시 기후 데이터프레임에 붙여줄 예정이다.
# 빈 리스트부터 만들어준다.
발생건수 = []

# 빈 리스트에 데이터를 담을 반복문을 써준다.
# 데이터프레임의 row 갯수 = date_list의 전체 길이
for index in range(len(date_list)) :
    일일발생건수 = sum(df.iloc[index, ])
    발생건수.append(일일발생건수)

In [160]:
# 발생건수는 list라, 데이터프레임에 붙이기 위해 데이터프레임으로 바꿔준다.
발생건수 = pd.DataFrame(발생건수) ; type(발생건수)

# merge함수를 쓰기 위해선, 기후 데이터프레임과 공통분모인 일시(날짜) 컬럼이 있어야 하며,
# 발생건수의 컬럼이름도 설정해주어야 하기에, 그 두 과정을 실시한다.
일시 = pd.DataFrame(date_list)
전체발생건수 = pd.concat([일시, 발생건수], axis = 1)

전체발생건수.columns = ["일시", "발생건수"] ; 전체발생건수

일시  발생건수
0     2018-01-01    19
1     2018-01-02     9
2     2018-01-03    23
3     2018-01-04    12
4     2018-01-05    14
...          ...   ...
1821  2022-12-27    17
1822  2022-12-28    13
1823  2022-12-29    19
1824  2022-12-30    18
1825  2022-12-31    22

[1826 rows x 2 columns]

In [146]:
# 이 데이터프레임을 csv로 저장한다.
전체발생건수.to_csv("서울시 전체 화재발생건수.csv", encoding ='cp949')

In [147]:
# 전체발생건수를 합칠 서울시 전체 기후 데이터를 불러온다
전체_기후 = pd.read_csv("서울시 전체 기후 데이터.csv", encoding = "cp949") ; 전체_기후

지점 지점명          일시  평균기온(°C)  최고기온(°C)  일강수량(mm)  평균 풍속(m/s)  \
0     108  서울  2018-01-01      -1.3       3.8       NaN         1.4   
1     108  서울  2018-01-02      -1.8       1.8       NaN         1.8   
2     108  서울  2018-01-03      -4.7      -0.4       NaN         2.2   
3     108  서울  2018-01-04      -4.7      -0.7       NaN         1.4   
4     108  서울  2018-01-05      -3.0       1.6       NaN         1.7   
...   ...  ..         ...       ...       ...       ...         ...   
1821  108  서울  2022-12-27      -2.6       3.3       NaN         1.8   
1822  108  서울  2022-12-28      -3.3       0.1       0.1         2.5   
1823  108  서울  2022-12-29      -2.9       2.1       0.0         1.7   
1824  108  서울  2022-12-30      -1.8       2.3       0.0         1.9   
1825  108  서울  2022-12-31      -1.2       2.1       0.0         1.4   

      평균 상대습도(%)  
0           39.1  
1           42.0  
2           42.3  
3           43.0  
4           48.4  
...          ...  
1821        69.8  
1822        58.1  
1823        56.3  
1824        65.6  
1825        65.5  

[1826 rows x 8 columns]

In [159]:
# 두 데이터프레임을 합치는데, 기후 데이터에는 비어있는 날짜가 존재한다.
# 그렇기에 merge함수를 사용하여, key인 일시 기준으로 두 데이터프레임을 결합해준다.
# 그 후. 이를 csv 파일로 저장해준다.
서울시_전체_기후_및_발생건수 = pd.merge(전체_기후, 전체발생건수) 
서울시_전체_기후_및_발생건수.to_csv("서울시 전체 기후 및 발생건수.csv", encoding = "cp949")

# 4.29일자 추가 크롤링
- 2017년 데이터 추가 크롤링
- 일별 데이터 > 월별 데이터로 바꾸기

In [9]:
# 반복문 전 작업

# 검색 전, 검색 칸의 내용을 지워주는 코드.
date_start = driver.find_element(By.NAME, "datePicker1s")
date_start.clear()

date_end = driver.find_element(By.NAME, "datePicker1e")
date_end.clear()

# 먼저 지역구 리스트부터 만든다.
region_list = ["강남구", "강동구", "강북구", "강서구", "관악구", "광진구",
               "구로구", "금천구", "노원구", "도봉구", "동대문구", "동작구",
              "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구",
              "양천구", "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"]

# 날짜 데이터를 2017년도로 한정짓는다.
date_2017 = pd.date_range(start='20170101', end='20171231')

date_list_2017 = date_2017.strftime("%Y-%m-%d").tolist()

# 화재 발생 건수를 담을 빈 리스트를 만든다
fire_counts = []

# 검색하기 누르는 코드
# 객체 생성
actions = ActionChains(driver)

# 검색하기 위치 지정
# searchItem > div.search-words > div.search-option > div.page-search > a:nth-child(1) > button 
elm_click = driver.find_element(By.CSS_SELECTOR,
                                "#searchItem > div.search-words > div.search-option > div.page-search > a:nth-child(1) > button")

# 검색하기 클릭
elm_click.click()

In [8]:
# 원하는 정보를 크롤링해오는 코드
# 2017년 데이터가 필요해서, 추가로 크롤링해 온다.

# 반복문 시작
for date in date_list_2017 :
    # 검색칸 중 시작날짜에 있는 내용을 지워준다.
    element_start = driver.find_element(By.NAME, "datePicker1s")
    element_start.clear()

    # 종료날짜도 똑같이 지워준다.
    element_end = driver.find_element(By.NAME, "datePicker1e")
    element_end.clear()
    
    # 시작날짜, 종료날짜에 처음부터 끝까지의 날짜 넣기
    date_start.send_keys(date) ; date_end.send_keys(date)
    
    # 검색 실행 > 데이터가 나온다.
    elm_click.click()
    
    # 클릭 이후 기다리면서 에러 방지하기
    # 기다려 주지 않으면, 날짜가 바뀌면서 정보가 바뀌는데, 그 속도가 컴퓨터의 속도보다 한참 느려 에러가 발생한다.
    time.sleep(1)
    
    # 이제 지역구 별로 화재발생 건수를 크롤링해온다.
    # f-string을 이용하여 선택자의 내부 숫자만 바꿀 예정인데, 이를 위해 선택자의 수를 파악할 필요가 있다.
    
    # 강남구 화재건수 : #grdViewer > table > tbody > tr:nth-child(4) > td:nth-child(2)
    # 중랑구 화재건수 : #grdViewer > table > tbody > tr:nth-child(28) > td:nth-child(2)
    # 이 상황에서, 4에서 28까지의 숫자를 고려하여 for 반복문을 설정해 준다.
    for i in range(4, 28 + 1) : # 4 ~ 28 = 강남구 ~ 중랑구
        fire_count = int(driver.find_element(By.CSS_SELECTOR, 
                                             f"#grdViewer > table > tbody > tr:nth-child({i}) > td:nth-child(2)").text)
        fire_counts.append(fire_count)
    

NameError: name 'elm_click' is not defined

In [156]:
# 위 크롤링 정보를 바탕으로 데이터프레임 생성
# 화재건수 데이터의 형태 변경
import numpy as np

# 열에는 지역구 이름 쓸것이므로, region_list에 맞게 재배열
fire_counts_2017 = np.array(fire_counts).reshape(-1, len(region_list))

df_2017 = pd.DataFrame(fire_counts_2017, columns=region_list, index=date_list_2017) ; df_2017

강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  도봉구  ...  성동구  성북구  \
2018-01-01    1    1    2    0    1    1    0    2    0    1  ...    2    0   
2018-01-02    0    0    0    1    1    0    1    0    0    0  ...    0    0   
2018-01-03    5    0    0    2    3    0    1    1    0    0  ...    0    1   
2018-01-04    0    0    0    0    0    0    0    0    1    0  ...    0    1   
2018-01-05    2    2    1    2    0    1    0    1    0    1  ...    0    0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2022-12-27    1    1    0    2    1    0    1    1    0    0  ...    1    1   
2022-12-28    2    0    1    1    0    0    0    0    0    0  ...    0    3   
2022-12-29    1    0    1    1    0    1    2    1    1    0  ...    1    0   
2022-12-30    2    0    0    0    2    0    0    0    0    0  ...    1    1   
2022-12-31    1    1    2    0    1    1    2    2    0    0  ...    0    1   

            송파구  양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
2018-01-01    2    0     1    0    0    0   2    1  
2018-01-02    0    1     1    0    1    1   0    1  
2018-01-03    4    0     0    0    0    1   0    1  
2018-01-04    3    0     1    1    0    2   0    2  
2018-01-05    0    0     0    0    1    2   0    0  
...         ...  ...   ...  ...  ...  ...  ..  ...  
2022-12-27    1    3     0    1    0    0   0    0  
2022-12-28    0    0     0    3    0    1   0    0  
2022-12-29    2    0     0    3    1    0   0    1  
2022-12-30    2    0     2    1    1    0   2    0  
2022-12-31    2    0     0    2    1    0   1    1  

[1826 rows x 25 columns]

In [163]:
# 결과값 csv 파일로 저장
# 디폴트값인 utf-8로 저장시 파일 깨짐 > 한글 인코딩 방식인 cp949 사용 (euc-kr도 깨짐)
df.to_csv("2017년 화재발생건수.csv", encoding='cp949')